In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dateutil.parser import parse
from time import sleep, time
from random import randint
import re
from IPython.core.display import clear_output
from warnings import warn
import numpy as np
from selenium import webdriver 
import cv2 
import pytesseract
from PIL import Image

In [2]:
def is_captcha_page(soup):
    captcha_text="Please verify you are human by typing the text below and clicking submit."
    page_info=soup.find('font',attrs={'class':"pagebodybold"})
    if page_info!=None and captcha_text in page_info.text:
        return True
    return False

In [3]:
def solve_captcha(company_link):
    driver = webdriver.Firefox('/home/udit/projects/chromiumm')
    driver.get(company_link)
    element = driver.find_element_by_id("imgCaptcha")
    element.screenshot('final.png')
    img = cv2.imread('final.png')
    captcha=pytesseract.image_to_string(img)
    
    captcha=re.sub(r'\W+', '', captcha)
    captcha_input = driver.find_element_by_xpath("//input[@id ='txtCaptcha']") 
    captcha_input.send_keys(captcha) 

    # submit button clicked
    driver.find_element_by_xpath("//input[@id ='btnRegiser']").click()
    html = driver.page_source
    soup = BeautifulSoup(html)
    driver.close()
    return soup

In [4]:
def try_filling(strr,company_info):
    try:
        global x
        eqn= "{0} = {1}".format("x",strr)
        x = np.nan
        exec(eqn,globals())
        return x
    except IndexError:
        return(np.nan)
    except ValueError:
        return(np.nan)

In [5]:
URL = "https://ibanknet.com/scripts/callreports/fiList.aspx?type=secbanks"
r = requests.get(URL)
soup = BeautifulSoup(r.content, 'html5lib')

In [6]:
companies=soup.find_all('a', attrs = {'class':'pagebody'}) 

In [7]:
all_companies=[]
error_links=[]

for company in companies[60:70]:
    company_link="https://ibanknet.com/"+company.get('href')
    try:
        c = requests.get(company_link)
        soup = BeautifulSoup(c.content, 'html5lib')

        while(is_captcha_page(soup)):
            soup=solve_captcha(company_link)

        company_info=[]

        for info in soup.find_all('td', attrs = {'class':'instinfo'}):
            company_info.append(info.text)


        company_inf={}
        company_inf["company_name"]=soup.find('font',attrs={'class':"blockuhilite"}).text.strip()
        company_inf["RSSD_ID"]=company_info[0].split()[company_info[0].split().index("RSSD")+2]
        company_inf['RefinedAddress']=company_info[1]
        company_inf['City']=company_info[4].split(",")[0].strip()
        company_inf['State']=company_info[4].split(",")[1].strip()
        company_inf['Pin']=try_filling('company_info[4].split(",")[2].strip()',company_info)
        company_inf['webAddress']=company_info[7].strip() if ('.com' in company_info[7].strip()) else np.nan
        company_inf['EntityType']=company_info[company_info.index('Entity Type : ')+1]
        company_inf['Incorporated']=company_info[company_info.index('Incorporated : ')+1]
        company_inf['SIC_Code']=company_info[company_info.index('SIC Code : ')+1]
        #company_inf['FederalRegulator']=company_info[company_info.index('Federal Regulator :')+1]
        company_inf['FiscalYearEnd']=company_info[company_info.index('Fiscal Year End : ')+1]
        company_inf['Regulator']=company_info[company_info.index('Regulator : ')+1]
        company_inf['FilerStatus']=company_info[company_info.index('Filer Status : ')+1]

        all_companies.append(company_inf)
        
    except Exception as e:
        error_links.append([company_link,str(e)])

In [8]:
all_compaies=pd.DataFrame(all_companies)

In [15]:
all_compaies.head()

,company_name,RSSD_ID,RefinedAddress,City,State,Pin,webAddress,EntityType,Incorporated,SIC_Code,FiscalYearEnd,Regulator,FilerStatus
0,ATLANTIC UNION BANKSHARES CORP,1971693,1051 EAST CARY STREET,RICHMOND,VA,23219,NaN,STATE COMMERCIAL BANKS,"VA, US",6022,1231,SEC,Large Accelerated
1,WASHINGTON FEDERAL INC,3065617,425 PIKE STREET,SEATTLE,WA,98101,NaN,NATIONAL COMMERCIAL BANKS,"WA, US",6021,930,SEC,Large Accelerated
2,CADENCE BANCORPORATION,4882802,2800 POST OAK BOULEVARD SUITE 3800,HOUSTON,TX,77056,NaN,NATIONAL COMMERCIAL BANKS,"DE, US",6021,1231,SEC,Large Accelerated
3,SOUTH STATE CORP,1133437,520 GERVAIS STREET,COLUMBIA,SC,29201,NaN,STATE COMMERCIAL BANKS,"SC, US",6022,1231,SEC,Large Accelerated
4,HOPE BANCORP INC,2961879,3200 WILSHIRE BLVD,LOS ANGELES,CA,90010,NaN,NATIONAL COMMERCIAL BANKS,,6021,1231,SEC,Large Accelerated


In [10]:
error_links

[]

In [11]:
#all_compaies.to_csv("ibankSeccanks.csv",index=False)

In [12]:
#len(all_companies)

In [13]:
#len(error_links)